# 텍스트 마이닝 02 - 텍스트 마이닝 주요 주제

<img width="40%" src="https://i.imgur.com/AydcsdT.png">
    
### 2017-2019 [FinanceData.KR]()

# 텍스트 마이닝 주요 주제
* Tokening
* Chunking
* TF-IDF (Term Frequency - Inverse Document Frequency) 단어 빈도와 역문서 빈도
* 코사인 유사도 (cosine similarity)
* LSI (Latent Semantic Indexing, 잠재 의미 색인),  LDA, HDP
* word2vec

# Tokening

In [1]:
text = 'The earth is a beautiful planet.'
tokens = text.split(' ')
tokens

['The', 'earth', 'is', 'a', 'beautiful', 'planet.']

# chunking

* 덩이짓기(Chunking, 청킹)
* 정보를 의미있는 묶음으로 만드는 것 (명사구)

# 문서의 단어 빈도와 여부

유사도(similarity): 비슷한 정도

### 문서에 단어 출현 빈도

| doc1 | doc2| doc3
-- | -- | -- | --
term1 | 0 | 0 | 2
term2 | 1 | 0 | 0
term3 | 0 | 1 | 3
term4 | 2 | 3 | 2
term5 | 3 | 2 | 0


### 문서에 단어 출현 여부(binary) 

| doc1 | doc2| doc3
-- | -- | -- | --
term1 | 1 | 0 | 0
term2 | 0 | 1 | 0
term3 | 0 | 0 | 1
term4 | 1 | 1 | 0
term5 | 1 | 1 | 1



#  코사인 유사도 (cosine similarity) 
문서 유사도 측정에 많이 사용

$$ similarity = \cos(\theta) = {A \cdot B \over \| A\| \| B\|} $$

In [0]:
import numpy as np

def my_cosine_similarity(A, B):
    return np.dot(A, B) / (np.sqrt(np.sum(A**2)) * np.sqrt(np.sum(B**2)))

In [3]:
doc1 = np.array([1, 0, 0, 1, 1])
doc2 = np.array([0, 1, 0, 1, 1])
doc3 = np.array([0, 0, 1, 0, 1])

print('(doc1, doc2):', my_cosine_similarity(doc1, doc2))
print('(doc1, doc3):', my_cosine_similarity(doc1, doc3))
print('(doc2, doc3):', my_cosine_similarity(doc2, doc3))

(doc1, doc2): 0.6666666666666667
(doc1, doc3): 0.40824829046386296
(doc2, doc3): 0.40824829046386296


# sklearn cosine_similarity

In [4]:
# 간편하게 cosine_similarity 매트릭스 구하기

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity([doc1, doc2, doc3])
similarity.round(4)

array([[1.    , 0.6667, 0.4082],
       [0.6667, 1.    , 0.4082],
       [0.4082, 0.4082, 1.    ]])

#  LSI (Latent Semantic Indexing), 잠재 의미 색인

### 배경
* 대상을 표현하는 방법은 다양하다 (동의어)
* 대부분의 단어는 여러 가지 의미를 가진다 (다중의미)

### LSI 핵심
* 1) Co-occurrence ▶SVD (특이값 분해)
* 2) Dimensionality Reduction ▶작은 특이값 제거

임의의 m×n의 행렬 M은 다음과 같이 분해될 수 있다


$$ \mathbf{M} = \mathbf{U} \boldsymbol{\Sigma} \mathbf{V}^* $$

<img src="https://i.imgur.com/uE5YqQd.gif" >

$\boldsymbol{\Sigma} $는 아래와 같이 구성되는데, diagonal entry들을 singular value(특이값)라고 한다. 

<img src="https://i.imgur.com/kENXTA1.gif" >




In [5]:
doc1 = np.array([1, 0, 0, 1, 1])
doc2 = np.array([0, 1, 0, 1, 1])
doc3 = np.array([0, 0, 1, 0, 1])

m = np.array(np.transpose([doc1, doc2, doc3]), dtype=np.float)
m

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 1., 0.],
       [1., 1., 1.]])

# Numpy svd()
Numpy의 linalg.svd()를 사용하면 간편하게 구할 수 있다

https://docs.scipy.org/doc/numpy/reference/routines.linalg.html

In [0]:
U, s, V = np.linalg.svd(m, full_matrices=True)

In [7]:
U.shape, s.shape, V.shape, 

((5, 5), (3,), (3, 3))

In [8]:
# U, s, V 값 확인
print('U', '-' * 80)
print(U.round(2))

print('s', '-' * 80)
print(s.round(2)) # 특이값

print('V', '-' * 80)
print(V.round(2))

U --------------------------------------------------------------------------------
[[-0.28  0.22  0.71 -0.47 -0.4 ]
 [-0.28  0.22 -0.71 -0.47 -0.4 ]
 [-0.16 -0.77 -0.    0.22 -0.57]
 [-0.56  0.44  0.    0.69 -0.18]
 [-0.71 -0.34 -0.   -0.22  0.57]]
s --------------------------------------------------------------------------------
[2.36 1.2  1.  ]
V --------------------------------------------------------------------------------
[[-0.66 -0.66 -0.37]
 [ 0.26  0.26 -0.93]
 [ 0.71 -0.71 -0.  ]]


In [0]:
# Reconstruction based on full SVD
S = np.zeros((5, 3))

In [10]:
S[:3, :3] = np.diag(s)
S

array([[2.35829447, 0.        , 0.        ],
       [0.        , 1.19935282, 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [11]:
# M=UΣV 인지 확인
np.allclose(m, np.dot(np.dot(U,S), V).round(2))

True

# 유사도 계산
1. 단어와 단어 사이 유사도: U x S 행렬의 행(row) 간의 유사도로 계산
2. 문서와 문서 사이 유사도: S x V 행렬의 컬럼(column) 간의 유사도로 계산
3. 단어와 문서 사이 유사도: U x S x V 행렬 자체

In [12]:
# 단어와 단어 사이 유사도:  U x S 행렬의 행(row) 간의 유사도로 계산
w = np.dot(U, S)

similarity = cosine_similarity(w)
similarity.round(4)

array([[ 1.    , -0.    ,  0.    ,  0.7071,  0.5774],
       [-0.    ,  1.    ,  0.    ,  0.7071,  0.5774],
       [ 0.    ,  0.    ,  1.    ,  0.    ,  0.5774],
       [ 0.7071,  0.7071,  0.    ,  1.    ,  0.8165],
       [ 0.5774,  0.5774,  0.5774,  0.8165,  1.    ]])

In [13]:
# 문서와 문서 사이 유사도:  S x V 행렬의 컬럼(column) 간의 유사도로 계산
d = np.dot(S, V)

similarity = cosine_similarity([d.T[0], d.T[1], d.T[2]])
similarity.round(4)

array([[1.    , 0.6667, 0.4082],
       [0.6667, 1.    , 0.4082],
       [0.4082, 0.4082, 1.    ]])

In [14]:
# 단어와 문서 사이 유사도: U x S x V 행렬 자체

np.dot(U, np.dot(S,V)).round(2)

array([[ 1., -0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  1.,  0.],
       [ 1.,  1.,  1.]])

# 차원축소

특이값 중 마지막(작은값)을 0으로 만들고, USV를 벡터를 구한다. 이 때 문서간의 유사도를 관찰

In [15]:
S

array([[2.35829447, 0.        , 0.        ],
       [0.        , 1.19935282, 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [16]:
# 작은 값을 0으로 만든다
print(S[2, 2])

S[2, 2] = 0
S

1.0


array([[2.35829447, 0.        , 0.        ],
       [0.        , 1.19935282, 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [17]:
np.dot(U, np.dot(S, V)).round(2)

array([[ 0.5,  0.5,  0. ],
       [ 0.5,  0.5, -0. ],
       [ 0. , -0. ,  1. ],
       [ 1. ,  1. ,  0. ],
       [ 1. ,  1. ,  1. ]])

doc1과 doc2의 term1, term2가 다른 단어지만 유사어 였다는 것을 알 수 있다 (잠재적인 의미 파악)

# 요약
* Tokening, Chunking
* TF-IDF (단어 빈도와 역문서 빈도)
* 코사인 유사도 (cosine similarity)
* LSI (Latent Semantic Indexing, 잠재 의미 색인)
 * 1) Co-occurrence ▶SVD (특이값 분해)
 * 2) Dimensionality Reduction ▶작은 특이값 제거
* 단어를 벡터(숫자)로

### 2017-2019 [FinanceData.KR]()